## Connect-Four (“4 em Linha”) Game using Minimax with Alpha-Beta Cuts

<img src="https://upload.wikimedia.org/wikipedia/en/7/79/Connect_4_Board_and_Box.jpg" width="250px" height="250px" align="right">

A board game is characterized by the type of board and tiles, the rules of movement of the pieces (operators/possible moves) and the finishing conditions of the game with the respective score/result.

The game called "Connect Four" in the English language version (“4 em Linha” in the Portuguese version - https://en.wikipedia.org/wiki/Connect_Four) is played on a vertical board of 7x6 squares (i.e., 7 squares wide and 6 squares high), by two players, to which are initially assigned 21 pieces to each (one of the players has white pieces and the other black pieces, or pieces "X" vs pieces "O").

The two players play alternately one of their pieces. The piece to be played is placed on the top of the board and slides either to the base of the board, or in a cell immediately above another one already occupied (see previous figure). The winner will be the player who manages to obtain a line of 4 pieces of its color/symbol horizontally, vertically, or diagonally. If the 42 pieces are played without any player getting a line, the final result will be a draw.

In [4]:
import random
import time
import numpy as np
from copy import deepcopy
import math

NUM_ROWS = 6
NUM_COLS = 7

class State:

    def __init__(self):
        # initialize the board info here and any additional variables
        self.board = np.zeros((NUM_ROWS, NUM_COLS)) # board initial state (all zeros)
        self.column_heights = [NUM_ROWS - 1] * NUM_COLS # useful to keep track of the index in which pieces should be inserted
        self.available_moves = list(range(7)) # list of playable columns (not full)
        self.player = 1
        self.winner = -1 # -1 - no winner (during game); 0 - draw; 1- player 1; 2 - player 2

    def move(self, column):
        # function that performs a move given the column number and returns the new state
        # do not forget to update the available moves list, column heights, pass the turn and check for winners
        state_copy = deepcopy(self)

        height = state_copy.column_heights[column]
        state_copy.column_heights[column] = height
        state_copy.board[height][column] = self.player

        if height == 0:
            state_copy.available_moves.remove(column)
        else:
            state_copy.column_heights[column] = height - 1

        state_copy.update_winner()
        state_copy.player = 3 - self.player # update player turn

        return state_copy

    def update_winner(self):
        # function that tests objective and update the winner accordingly
        # sholud return 1, 2 or 0 (draw)
        if(len(self.available_moves) == 0):
            self.winner = 0
            return;
        if(self.count_lines(4,1) >= 1):
            self.winner = 1
        elif (self.count_lines(4,2) >=1):
            self.winner = 2
        else:
            self.winner = -1


    def check_line(self, n, player, values):
        num_pieces = sum(list(map(lambda val: val == player, values)))
        if n == 4:
            return num_pieces == 4
        if n == 3:
            if(num_pieces != 4):
              num_empty_spaces = sum(list(map(lambda val: val == 0, values)))
              return num_pieces == 3 and num_empty_spaces == 1
            return 0


    # c1) c2)
    def count_lines(self, n, player):
        num_lines = 0
        for row in range(NUM_ROWS):
          for col in range(NUM_COLS):
              if col < NUM_COLS - 3 and self.check_line(n, player, [self.board[row][col], self.board[row][col+1], self.board[row][col+2], self.board[row][col+3]]):
                  num_lines += 1
              if row < NUM_ROWS - 3 and self.check_line(n, player, [self.board[row][col], self.board[row+1][col], self.board[row+2][col], self.board[row+3][col]]):
                  num_lines += 1
              for diffx in [-1, 1]:
                diffy = -1
                if row < NUM_ROWS - 3 and col < NUM_COLS - 3:
                  positions = [
                    self.board[row][col],
                    self.board[row+diffx][col+diffy],
                    self.board[row+(diffx*2)][col+(diffy*2)],
                    self.board[row+(diffx*3)][col+(diffy*3)]
                  ]
                  if self.check_line(n, player, positions):
                    num_lines +=1
        return num_lines
    # c3)
    def central(self, player):
        count = 0
        for row in range(NUM_ROWS):
          if(self.board[row][3] == player):
            count += 2
          if(self.board[row][2] == player):
            count+=1
          if(self.board[row][4] == player):
            count+=1
        return count

    def isEndState(self):
      return self.winner != -1

class ConnectFourGame:

    def __init__(self, player_1_ai, player_2_ai):
        self.state = State()
        self.player_1_ai = player_1_ai
        self.player_2_ai = player_2_ai

    def start(self, log_moves = False):
        self.state = State()
        while True:

            if self.state.player == 1:
                self.player_1_ai(self)
            else:
                self.player_2_ai(self)

            if log_moves:
                print(self.state.board)

            if self.state.winner != -1:
                break

        if self.state.winner == 0:
            print("End of game! Draw!")
        else:
            print(f"End of game! Player {self.state.winner} wins!")
        # print(self.state.board)
        # print()

    def run_n_matches(self, n, max_time = 3600, log_moves = False):
        start_time = time.time()

        results = [0, 0, 0] # [draws, player 1 victories, player 2 victories]

        # Your Code Here
        for _ in range(n):
          self.start()
          results[self.state.winner] += 1

        print("\n=== Elapsed time: %s seconds ===" % (int(time.time() - start_time)))
        print(f"  Player 1: {results[1]} victories")
        print(f"  Player 2: {results[2]} victories")
        print(f"  Draws: {results[0]} ")
        print("===============================")

"""
    Heuristic functions - e)
"""

def evaluate_f1(state):
    return state.count_lines(4, 1) - state.count_lines(4, 2)

def evaluate_f2(state):
    return (state.count_lines(4, 1) - state.count_lines(4, 2)) * 100 + state.count_lines(3, 1) - state.count_lines(3, 2)

def evaluate_f3(state):
    return 100 * evaluate_f1(state) + state.central(1) - state.central(2)

def evaluate_f4(state):
    return 5 * evaluate_f2(state) + evaluate_f3(state)

"""
    Move selection methods
"""

def execute_random_move(game):
    move = random.choice(game.state.available_moves)
    game.state = game.state.move(move)

def execute_minimax_move(evaluate_func, depth):
  def execute(game):
    evaluate,state = minimax(game.state,depth,-math.inf,math.inf,0,True,evaluate_func)
    game.state = state
  return execute


def minimax(state, depth, alpha, beta, maximizing, player, evaluate_func):
  if depth == 0 or state.isEndState(): return (evaluate_func(state),state)

  if player:
    maxEval = -math.inf
    for move in state.available_moves:
      move_state = state.move(move)
      (evaluation,new_state) = minimax(move_state,depth-1, alpha, beta,True,False,evaluate_func)
      maxEval = max(maxEval, evaluation)
      alpha = max(alpha, evaluation)
      if beta <= maxEval: break
    return (maxEval,new_state)

  minEval = math.inf
  for move in state.available_moves:
    move_state = state.move(move)
    (evaluation,new_state) = minimax(move_state,depth-1, alpha, beta,True,True,evaluate_func)
    minEval = min(minEval, evaluation)
    alpha = min(alpha, evaluation)
    if minEval <= alpha: break
  return (minEval,new_state)


In [ ]:
# Random vs random
game = ConnectFourGame(execute_random_move, execute_random_move)
game.run_n_matches(10, 120, False)

End of game! Player 2 wins!
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [2. 2. 2. 2. 1. 0. 0.]
 [1. 2. 1. 2. 1. 0. 1.]]

End of game! Player 1 wins!
[[0. 0. 0. 0. 0. 0. 0.]
 [2. 0. 1. 0. 0. 0. 0.]
 [1. 1. 1. 0. 0. 0. 0.]
 [2. 1. 1. 0. 0. 0. 0.]
 [2. 1. 2. 2. 0. 1. 2.]
 [1. 1. 2. 2. 2. 1. 2.]]

End of game! Player 2 wins!
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 1. 2.]
 [0. 0. 0. 2. 0. 2. 2.]
 [0. 0. 1. 1. 0. 1. 2.]
 [0. 0. 2. 1. 0. 1. 2.]]

End of game! Player 1 wins!
[[0. 2. 0. 0. 0. 1. 1.]
 [0. 1. 2. 0. 2. 1. 2.]
 [0. 1. 1. 1. 1. 1. 2.]
 [2. 2. 2. 1. 1. 2. 2.]
 [2. 2. 1. 2. 2. 1. 1.]
 [1. 1. 1. 2. 1. 2. 2.]]

End of game! Player 2 wins!
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 1. 0. 0.]
 [2. 0. 2. 2. 2. 2. 1.]
 [1. 0. 2. 1. 1. 2. 1.]]

End of game! Player 2 wins!
[[2. 0. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 1. 0.]
 [2. 0. 1. 0. 0. 1. 0.]
 [2. 1. 2. 2. 2. 2. 2.]
 [1. 2

In [2]:
# Minimax (f1, depth = 2) vs random
game = ConnectFourGame(execute_minimax_move(evaluate_f1, 2), execute_random_move)
game.run_n_matches(10, 120)

End of game! Player 1 wins!
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1.]
 [2. 0. 0. 0. 0. 0. 1.]
 [2. 0. 0. 0. 0. 0. 1.]
 [2. 0. 0. 0. 0. 0. 1.]]

End of game! Player 1 wins!
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1.]
 [2. 0. 0. 0. 0. 0. 1.]
 [2. 0. 0. 0. 0. 0. 1.]
 [2. 0. 0. 0. 0. 0. 1.]]

End of game! Player 1 wins!
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1.]
 [2. 0. 0. 0. 0. 0. 1.]
 [2. 0. 0. 0. 0. 0. 1.]
 [2. 0. 0. 0. 0. 0. 1.]]

End of game! Player 1 wins!
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1.]
 [2. 0. 0. 0. 0. 0. 1.]
 [2. 0. 0. 0. 0. 0. 1.]
 [2. 0. 0. 0. 0. 0. 1.]]

End of game! Player 1 wins!
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1.]
 [2. 0. 0. 0. 0. 0. 1.]
 [2. 0. 0. 0. 0. 0. 1.]
 [2. 0. 0. 0. 0. 0. 1.]]

End of game! Player 1 wins!
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1.]
 [2. 0. 0. 0. 0. 0. 1.]
 [2. 0

In [3]:
# Minimax (f2, depth = 2) vs random
game = ConnectFourGame(execute_minimax_move(evaluate_f2, 2), execute_random_move)
game.run_n_matches(10, 120)

End of game! Player 1 wins!
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1.]
 [2. 0. 0. 0. 0. 0. 1.]
 [2. 0. 0. 0. 0. 0. 1.]
 [2. 0. 0. 0. 0. 0. 1.]]

End of game! Player 1 wins!
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1.]
 [2. 0. 0. 0. 0. 0. 1.]
 [2. 0. 0. 0. 0. 0. 1.]
 [2. 0. 0. 0. 0. 0. 1.]]

End of game! Player 1 wins!
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1.]
 [2. 0. 0. 0. 0. 0. 1.]
 [2. 0. 0. 0. 0. 0. 1.]
 [2. 0. 0. 0. 0. 0. 1.]]

End of game! Player 1 wins!
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1.]
 [2. 0. 0. 0. 0. 0. 1.]
 [2. 0. 0. 0. 0. 0. 1.]
 [2. 0. 0. 0. 0. 0. 1.]]

End of game! Player 1 wins!
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1.]
 [2. 0. 0. 0. 0. 0. 1.]
 [2. 0. 0. 0. 0. 0. 1.]
 [2. 0. 0. 0. 0. 0. 1.]]

End of game! Player 1 wins!
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1.]
 [2. 0. 0. 0. 0. 0. 1.]
 [2. 0

In [5]:
# Minimax (f3, depth = 2) vs random
game = ConnectFourGame(execute_minimax_move(evaluate_f3, 2), execute_random_move)
game.run_n_matches(10, 120)

End of game! Player 1 wins!
End of game! Player 1 wins!
End of game! Player 1 wins!
End of game! Player 1 wins!
End of game! Player 1 wins!
End of game! Player 1 wins!
End of game! Player 1 wins!
End of game! Player 1 wins!
End of game! Player 1 wins!
End of game! Player 1 wins!

=== Elapsed time: 1 seconds ===
  Player 1: 10 victories
  Player 2: 0 victories
  Draws: 0 


In [ ]:
# Minimax (f4, depth = 2) vs random
game = ConnectFourGame(execute_minimax_move(evaluate_f4, 2), execute_random_move)
game.run_n_matches(10, 120)

In [6]:
# Minimax (f1, depth = 2) vs Minimax (f4, depth = 2)
game = ConnectFourGame(execute_minimax_move(evaluate_f1, 2), execute_minimax_move(evaluate_f4, 2))
game.run_n_matches(5, 120)

End of game! Player 1 wins!
End of game! Player 1 wins!
End of game! Player 1 wins!
End of game! Player 1 wins!
End of game! Player 1 wins!

=== Elapsed time: 0 seconds ===
  Player 1: 5 victories
  Player 2: 0 victories
  Draws: 0 


In [7]:
# Minimax (f4, depth = 2) vs Minimax (f4, depth = 4)
game = ConnectFourGame(execute_minimax_move(evaluate_f4, 2), execute_minimax_move(evaluate_f4, 4))
game.run_n_matches(3, 240, True)

End of game! Player 2 wins!
End of game! Player 2 wins!
End of game! Player 2 wins!

=== Elapsed time: 1 seconds ===
  Player 1: 0 victories
  Player 2: 3 victories
  Draws: 0 


In [8]:
state = State()
state.board = [
    [0,0,0,0,0,0,0],
    [0,0,2,0,0,0,0],
    [0,0,0,2,0,0,0],
    [0,0,0,0,2,0,0],
    [0,0,1,0,0,2,0],
    [1,1,1,0,0,0,0],
]
state.update_winner()
print(state.winner)


-1
